In [27]:
from qiskit import Aer, QuantumCircuit, transpile, assemble
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms import QSVC
from qiskit.utils import QuantumInstance
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from qiskit_machine_learning.kernels import FidelityQuantumKernel

In [28]:
# Load the data credit card dataset
data = pd.read_csv('creditcard.csv')
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [29]:
data0 = data[data['Class'] == 0].sample(800)
data0
#take 492 class=1 data
data1 = data[data['Class'] == 1]
#data0+data1
data = pd.concat([data0, data1])
data        
#shuffle randomly 'data dataframe'
data = data.sample(frac=1).reset_index(drop=True)
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,41238.0,1.118915,-1.070323,0.193135,-0.593994,-1.004806,0.080457,-0.645465,0.191352,-0.601172,...,-0.527350,-1.269353,0.074256,-0.345406,-0.037261,0.883660,-0.068244,0.006346,108.35,0
1,34256.0,0.539276,1.554890,-2.066180,3.241617,0.184736,0.028330,-1.515521,0.537035,-1.999846,...,0.371773,0.111955,-0.305225,-1.053835,0.771175,0.240878,0.418435,0.232170,19.02,1
2,111642.0,-0.883422,0.106626,1.894251,-2.075304,-1.305249,1.452857,-2.038377,-2.743941,0.085966,...,3.144303,-0.691187,-0.153458,0.768025,1.611231,-0.068653,0.219282,0.147812,32.82,0
3,127470.0,2.171072,-1.635998,-1.427730,-1.869299,-0.631935,0.375072,-1.089417,-0.033127,-1.487704,...,-0.158820,-0.153029,0.068781,-0.349296,-0.111412,-0.200058,-0.005428,-0.045760,99.95,0
4,7526.0,0.008430,4.137837,-6.240697,6.675732,0.768307,-3.353060,-1.631735,0.154612,-2.795892,...,0.364514,-0.608057,-0.539528,0.128940,1.488481,0.507963,0.735822,0.513574,1.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1287,76826.0,-6.616293,3.563428,-7.058901,4.284346,-5.096299,-1.768618,-4.937554,2.748460,-3.796760,...,1.215976,0.041178,-1.059098,0.275662,0.057425,-0.265838,-0.514637,0.388590,254.76,1
1288,123153.0,-0.013156,0.634087,-0.219137,-2.316159,0.893955,-1.909816,1.823732,-0.805116,0.275434,...,0.155854,0.747449,-0.427911,0.118840,0.242977,-0.053724,-0.066016,-0.087740,15.20,0
1289,7474.0,-1.041752,0.373701,2.451208,-0.799679,-0.594094,0.209621,0.208222,0.251656,1.434598,...,0.001596,0.205492,-0.150904,-0.004195,0.288841,1.015727,0.159771,0.104350,94.93,0
1290,64443.0,1.079524,0.872988,-0.303850,2.755369,0.301688,-0.350284,-0.042848,0.246625,-0.779176,...,-0.023255,-0.158601,-0.038806,-0.060327,0.358339,0.076984,0.018936,0.060574,0.00,1


In [30]:
# Extract features and labels
X = data.iloc[:, 1:-1].values
y = data.iloc[:, -1].values
X.shape

(1292, 29)

In [31]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [33]:
# Define the quantum feature map
feature_map = ZZFeatureMap(feature_dimension=X_train.shape[1], reps=2, entanglement='linear')


In [34]:
qkernel = FidelityQuantumKernel(feature_map=feature_map)

In [35]:
# Create the QSVC instance
qsvc = QSVC(quantum_kernel=qkernel)

In [36]:
# Run the QSVC on a quantum simulator (Aer backend)
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=42, seed_transpiler=42)

C:\Users\HP\AppData\Local\Temp\ipykernel_18044\2111074854.py:3: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=42, seed_transpiler=42)


In [39]:
# Train the QSVC
result = qsvc.fit(X_train, y_train)

In [ ]:
score=qsvc.score(X_test, y_test)
print(f"QSVC classification test score: {score}")

In [ ]:
# Evaluate the accuracy
accuracy = accuracy_score(y_test, score)
print(f"Classification Accuracy: {accuracy}")